# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
from IPython.display import display, Math

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

The **monopoly pricing model** is a basic model in microeconomics. We look at a monopoly on a market, looking to maximize their own profit. Given the demand for the consumers of the market, the monopoly can set a quantity which implies a ceratin price for the quantity sold, all adding up to the biggest profit possible.

First, we consider the **basic monopoly pricing model**:

1. $P$ is the price set by the monopolist.
2. $Q$ is the quantity demanded at price P.
3. $C(Q)$ is the cost function dependens on the quantity produced.

Thereby, the **demand function** is ***linear*** and given by:
$$Q=a-bP$$

Where a is the maximum quantity that could be sold at P=0 while b can be interpreted as the sensititivy of the the quantity demanded to changes in price. The inverse is therefore:
$$P(Q)=\frac{1}{b}(a-Q)$$

Furthermore, the **cost function** is ***linear*** and given by:
$$C(Q) = cQ+F$$

Where c is the variable cost per unit of production and F is the fixed cost.

Funally, the **Revenue** is given by:
$$TR(Q) = P(Q) \cdot Q$$

Thereby, the **Profit** is given by:
$$\pi (Q) = TR(Q)-C(Q)$$


## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

First, i start off by solving the model analytically using *sumpy*:


In [2]:
# Define the symbols
Q = sm.symbols('Q')
a, b, c, F = sm.symbols('a b c F')
pi_symbol = sm.symbols('pi')

# Define the inverse demand function
P = (a - Q) / b

# Define the cost function
C = c * Q + F

# Define the revenue function
TR = P * Q

# Define the profit function
profit = TR - C

# Create an equation for the first-order condition by differentiating the profit function with respect to Q
profit_eq = sm.Eq(sm.diff(profit, Q), 0)

# Solve the first-order condition for Q to find the quantity that maximizes profit
Q_star = sm.solve(profit_eq, Q)[0]

# Calculate the optimal price by substituting Q_star into the inverse demand function
P_star = P.subs(Q, Q_star)

# Calculate the optimal profit by substituting Q_star into the profit function
profit_star = profit.subs(Q, Q_star)

profit_star

-F - c*(a/2 - b*c/2) + (a/2 - b*c/2)*(a/2 + b*c/2)/b

I make a function where i turn my functions *Q_star*, *P_star* and *profit_star* into python functions using `lambdify` and then print $Q^*$, $P^*$ and $\pi^*$ :

In [3]:
ss_Q = sm.lambdify((a, b, c), Q_star, "numpy")
ss_P = sm.lambdify((a, b, c), P_star, "numpy")
ss_profit = sm.lambdify((a, b, c, F), profit_star, "numpy")

def optimum(a,b,c, F):
    #We make the python functions:
    optimal_Q = ss_Q(a, b, c)
    optimal_P = ss_P(a, b, c)
    optimal_profit = ss_profit(a, b, c, F)
    
    # Print the results formatted as requested
    print("Optimal quantity Q is equal to: {:.2f}".format(optimal_Q))
    print("Optimal price P is equal to: {:.2f}".format(optimal_P))
    print(f"Optimal profit \u03C0 is equal to: {optimal_profit:.2f}")

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# **VI MANGLER NUMMERISK LØSNING. ANER IKKE HVORDAN MAN GØR DET, VI MÅ SPØRGE JONATHAN**
OG SÅ SYNES JEG FAKTISK OGSÅ DET KUNNE VÆRE RET FLEX AT LAVE ET STANDARD MR, MC OG AC

# Further analysis

# INTERACTIVE 3D PLOT OF Q, P AND PROFIT

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive

def plot_optimal_values(a, b, c, F):
    optimal_Q = ss_Q(a, b, c)
    optimal_P = ss_P(a, b, c)
    optimal_profit = ss_profit(a, b, c, F)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Set fixed limits for each axis
    ax.set_xlim(0, 20)  # Adjust the limits as needed
    ax.set_ylim(0, 20)  # Adjust the limits as needed
    ax.set_zlim(-50, 50)  # Adjust the limits as needed
    
    ax.scatter(optimal_Q, optimal_P, optimal_profit, color='r')
    
    # Plot lines from the red dot to each axis
    ax.plot([optimal_Q, optimal_Q], [0, optimal_P], [optimal_profit, optimal_profit], color='b', linestyle='--')  # Line to x-axis
    ax.plot([0, optimal_Q], [optimal_P, optimal_P], [optimal_profit, optimal_profit], color='g', linestyle='--')  # Line to y-axis
    ax.plot([optimal_Q, optimal_Q], [optimal_P, optimal_P], [0, optimal_profit], color='purple', linestyle='--')  # Line to z-axis
    
    # Annotate the plot with axis values
    ax.text(optimal_Q, 0, 0, f'Q: {optimal_Q:.2f}', color='b', fontsize=10)
    ax.text(0, optimal_P, 0, f'P: {optimal_P:.2f}', color='g', fontsize=10)
    ax.text(0, 0, optimal_profit, f'π: {optimal_profit:.2f}', color='purple', fontsize=10)
    
    ax.set_xlabel('Optimal Quantity (Q)')
    ax.set_ylabel('Optimal Price (P)')
    ax.set_zlabel('Optimal Profit (π)')
    
    plt.title('3D View of Optimal Q, P, and π')
    plt.show()

# Set up interactive widgets
interactive_plot = interactive(plot_optimal_values, a=(0.1, 50.0, 0.1), b=(0.1, 5.0, 0.1), c=(0.1, 5.0, 0.1), F=(0.1, 50.0, 0.1))
interactive_plot


interactive(children=(FloatSlider(value=25.000000000000004, description='a', max=50.0, min=0.1), FloatSlider(v…

# FURTHER ANALYSIS 1: ALTERNATIVE DEMAND AND COST FUNCTIONS

When solving the model, we implied the most basic versions of demand and cost functions. Therefore, the results are also pretty easy to interpret and solve. Because of that, we will try to change the analysis by implementing an exponential demand function and a quadratic cost function:

1. **Exponential Demand Function**:
   $$
   Q = a \cdot e^{-bP}
   $$
   Here, $a$ is the maximum potential demand when the price $P$ is very low, and $b$ represents the sensitivity of the quantity demanded to changes in price.

2. **Inverse Demand Function**:
   $$
   P(Q) = -\frac{1}{b} \log\left(\frac{Q}{a}\right)
   $$
   This function is derived by solving the exponential demand equation for $P$.

3. **Quadratic Cost Function**:
   $$
   C(Q) = cQ + dQ^2 + F
   $$
   In this function, $c$ is the variable cost per unit, $d$ is the coefficient for the quadratic term indicating increasing marginal costs, and $F$ is the fixed cost.

4. **Total Revenue (TR)**:
   $$
   TR(Q) = P(Q) \cdot Q = \left(-\frac{1}{b} \log\left(\frac{Q}{a}\right)\right) \cdot Q
   $$
   This represents the total income received from selling $Q$ units at price $P(Q)$.

5. **Profit Function**:
   $$
   \pi(Q) = TR(Q) - C(Q) = \left(-\frac{1}{b} \log\left(\frac{Q}{a}\right)\right) \cdot Q - (cQ + dQ^2 + F)
   $$
   Profit is calculated by subtracting the total cost from the total revenue.


In [10]:
import sympy as sm
from scipy import optimize

def solve_model(a, b, c, d, F):
    # Define symbols
    Q = sm.symbols('Q')
    
    # Exponential demand function
    P = -sm.log(Q/a) / b
    
    # Quadratic cost function
    C = c * Q + d * Q**2 + F
    
    # Revenue function
    TR = P * Q
    
    # Profit function
    profit = TR - C
    
    # First-order condition: differentiate profit with respect to Q and set to 0
    profit_eq = sm.Eq(sm.diff(profit, Q), 0)
    
    # Solve the first-order condition for Q to find the quantity that maximizes profit
    Q_star = sm.solve(profit_eq, Q)[0]
    
    # Calculate the optimal price by substituting Q_star into the inverse demand function
    P_star = P.subs(Q, Q_star)
    
    # Calculate the optimal profit by substituting Q_star into the profit function
    profit_star = profit.subs(Q, Q_star)

    # Print the results
    print("Optimal quantity Q is equal to:", Q_star.evalf())
    print("Optimal price P is equal to:", P_star.evalf())
    print("Optimal profit π is equal to:", profit_star.evalf())
    
    return Q_star.evalf(), P_star.evalf(), profit_star.evalf()

In [11]:
def plot_optimal_values(a, b, c, d, F):
    Q_star, P_star, profit_star = solve_model(a, b, c, d, F)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Set fixed limits for each axis
    ax.set_xlim(0, 50)
    ax.set_ylim(0, 50)
    ax.set_zlim(0, 500)
    
    ax.scatter(Q_star, P_star, profit_star, color='r', s=100)
    
    ax.set_xlabel('Quantity (Q)')
    ax.set_ylabel('Price (P)')
    ax.set_zlabel('Profit (π)')
    
    ax.set_title('Optimal Q, P, and Profit')
    
    plt.show()

# Set up interactive widgets
interactive_plot = interactive(plot_optimal_values, 
                               a=(0.1, 100.0, 1), 
                               b=(0.0, 0.5, 0.05), 
                               c=(0.1, 10.0, 0.1), 
                               d=(0.01, 1.0, 0.1), 
                               F=(0.1, 100.0, 5))
interactive_plot

interactive(children=(FloatSlider(value=49.1, description='a', min=0.1, step=1.0), FloatSlider(value=0.25, des…

# **FURTHER ANALYSIS 2: DUOPOLY INSTEAD OF MONOPOLY**

# Conclusion

Add concise conclusion. 

## **SPØRGSMÅL OG TO DO TIL JONATHAN:**
- **NUMMERISK OPTIMERING for gamle model (SPØRGSMÅL)**
- **Duopol**